# Social bubbles with Covid

This notebook demonstrates the use of the simulator on the network with bubbles.

### Imports and setup

In [1]:
import numpy as np
import simulation_functions_new as DCT
from utils import temporal_graph_load
from tqdm.notebook import tqdm
import pickle
import os

%load_ext autoreload
%autoreload 2

### Define the parameters of the simulation

In [2]:
eps_I_list = [0.1,0.25,0.4]
eps_T_list = [0,0.01,0.1,0.4]

In [3]:
net = 'synth' 

In [4]:
K = 10
nb_ideas = 20
k_threshold = np.full(nb_ideas,K)
#k_threshold = [10, 8, 12, 12, 8, 8, 10, 12, 10, 12, 8, 12, 12, 10, 8, 8, 10, 12, 10, 8] #mix1
#k_threshold = [10, 8, 12] #mix2

In [5]:
PARAMETERS = {'temporal_gap' : 3600*2,              # Temporal gap between static networks (sec)
              'nb_nodes' : 850,
              'nb_clusters' : 10,                   # Nb of clusters in networks
              'cluster_size' : 85,                  # Cluster size
              'nb_ideas' : len(k_threshold),                # Nb of ideas
              'nb_links' : 625,                     # nb of links in networks
              'memory_contacts' : 7,                # Tracing memory (days)
              'max_time_quar' : 10,                 # Quarantine duration (days)
              'max_time_iso' : 25,                  # Isolation duration (days)
              'eps_Is' : eps_I_list,                # Isolation effectivity
              'eps_Ts' : eps_T_list,                # Tracing effectivity
              'recov_time' : 25,                    # recovery time
              'k_threshold' : k_threshold,          # Knowledge spreading parameter
              'times' : 200,                      # Number of repetition of the simulation
              'seed' : 11,                          # Random seed 
              'symptomatics' : 0.8,                 # Fraction of symptomatic individuals 
              'testing' : 0.0,                      # Fraction of asymptomatics who are detected via random testing
              'net' : net,
              'store':{'to_store': True,            # Save the results?
                       'path_to_store':'RESULTS/'}} # Target folder to save the results

In [6]:
5*136

680

### Parameters and graph length

In [7]:
nb_clusters = PARAMETERS['nb_clusters']
nb_nodes = PARAMETERS['nb_nodes']
cluster_size = nb_nodes/nb_clusters
if PARAMETERS['cluster_size'] != cluster_size:
    print('ERRORE!!!')
L = PARAMETERS['nb_links']

In [8]:
def update_store_path(path_to_store,eps_I,eps_T):
    if all(el == PARAMETERS['k_threshold'][0] for el in PARAMETERS['k_threshold']): # True if all K are the same
        name='/epsI_' + str(eps_I)+'_epsT_'+str(eps_T) + '_kThresh_'+str(PARAMETERS['k_threshold'][0])+'/'
    else:
         name='/epsI_' + str(eps_I)+'_epsT_'+str(eps_T) + '_kThresh_mix2/'
    #print(path_to_store)
    #print(name)
    path_to_store += name

    if not os.path.exists(path_to_store):
        os.makedirs(path_to_store)
    return path_to_store

## Run the simulations and store the results

In [9]:
# Initialize the random seed
np.random.seed(PARAMETERS['seed'])

In [10]:
rounds = True
round_days = 10

In [11]:
#for p_intra in [0.00483,0.00621,0.00725,0.00781,0.00806,0.00838,0.00849,0.00854,0.00858,0.0086,0.00863,0.00865]: #5 clusters
#for p_intra in [0.00621,0.0121,0.01485,0.01393,0.01609,0.01655,0.01679,0.01694,0.01704,0.01717,0.01727]: #10 clusters
#for p_intra in [0.00725,0.02562,0.02981,0.03154,0.03247,0.03306,0.03347,0.03399,0.03443]: #20 clusters


#for p_intra in [0.00621,0.01203,0.01481,0.01605,0.01650,0.01674]: 
for p_intra in [0.01203]: 
    print('p_intra',p_intra)
    p_inter = 2*L/(nb_clusters*(nb_clusters-1)*cluster_size**2) - (cluster_size-1)*p_intra/((nb_clusters-1)*cluster_size)
    graph_name = '%dnodes/%dclusters_%dnodes/temporal_net_p_intra%.2E_p_inter%.2E'%(nb_nodes,nb_clusters,cluster_size,p_intra,p_inter)
    if rounds:
        graph_name += '_rounds%ddays'%round_days
        
    graphs = temporal_graph_load(graph_name)
    path_to_store = PARAMETERS['store']['path_to_store'] + graph_name
    
    #for eps_I in eps_I_list:
    for eps_I in [0.25]:
        for eps_T in eps_T_list[:1]:
        
            print('eps_I',eps_I,'eps_T',eps_T)
            
            path_to_store_upd = update_store_path(path_to_store,eps_I,eps_T)
            
            print(path_to_store_upd)

            # Print some report
            print('-' * 100)
            print('Running simulation with')
            print('       |_ eps_T             : %4.2f'   % eps_T)
            print('       |_ eps_I             : %.3f'   % eps_I)
            print('       |_ p_intra             : %.5f'   % p_intra)
            print('')


            # Repeat the simulation for a fixed configuration             
            for i in tqdm(range(PARAMETERS['times'])):
                
                # Initialize the simulation object
                dct = DCT.DigitalContactTracing(path_to_store_upd,
                                                graphs=graphs,
                                                PARAMETERS = PARAMETERS,
                                                eps_I=eps_I,
                                                eps_T=eps_T,
                                                realization=i)
                # Run the actual simulation
                res = dct.simulate()


p_intra 0.01203


FileNotFoundError: [Errno 2] No such file or directory: 'Graphs/850nodes/10clusters_85nodes/temporal_net_p_intra1.20E-02_p_inter6.01E-04_rounds10days/num_snapshots.txt'

In [ ]:
eps_I

### Load the results after the simulation

In [ ]:
nb_nodes = nb_clusters*cluster_size

In [ ]:
p_intra = 0.00483 #5 clusters
p_inter = 2*L/(nb_clusters*(nb_clusters-1)*cluster_size**2) - (cluster_size-1)*p_intra/((nb_clusters-1)*cluster_size)
graph_name = '%dclusters_%dnodes/temporal_net_p_intra%.2E_p_inter%.2E'%(nb_clusters,cluster_size,p_intra,p_inter)
if rounds:
    graph_name += '_rounds%ddays'%round_days
path = PARAMETERS['store']['path_to_store'] + graph_name
print(path)
eps_I = eps_I_list[0]
eps_T = eps_T_list[0]
path = update_store_path(path,eps_I,eps_T)
print(path)

for real in range(PARAMETERS['times']): # for each realization
    # load act_inf:
    globals()['r%d_act_inf'%real] = np.load(path + '%d_act_inf.npy'%real)
    # load isolated:
    globals()['r%d_isolated'%real] = np.load(path + '%d_isolated.npy'%real)
    # load recovered:
    globals()['r%d_recovered'%real] = np.load(path + '%d_recovered.npy'%real)
    if eps_T > 0:
        # load quarantined:
        globals()['r%d_quarantined'%real] = np.load(path + '%d_quarantined.npy'%real)

    # load ideas_per_node and nodes_per_idea at all times:
    globals()['r%d_ideas_per_node'%real] = np.load(path + '%d_ideas_per_node.npy'%real)
    globals()['r%d_nodes_per_idea'%real] = np.load(path + '%d_nodes_per_idea.npy'%real)
    
    # ideas acquired at time t:
    #for t in range(len(graphs)):
    for t in range(11):
        globals()['r%d_ideas_per_node_time_%d'%(real,t)] = r0_ideas_per_node[t*nb_nodes:(t+1)*nb_nodes] # time t
        globals()['r%d_nodes_per_idea_time_%d'%(real,t)] = r0_nodes_per_idea[t*nb_ideas:(t+1)*nb_ideas] # time t

        # load effective edgelist:
        globals()['r%d_eff_edgelist_time_%d'%(real,t)] = np.load(path + 'eff_edgelist/%d_eff_edgelist_time_%d.npy'%(real,t))



In [ ]:
real = 1
t = 10
globals()['r%d_eff_edgelist_time_%d'%(real,t)] = np.load(path + 'eff_edgelist/%d_eff_edgelist_time_%d.npy'%(real,t))

In [ ]:
r0_eff_edgelist_time_10

In [ ]:
r0_ideas_per_node_time_1

In [ ]:
r0_nodes_per_idea_time_1

In [ ]:
r0_eff_edgelist_time_4

In [ ]:
################

In [ ]:
nb_nodes = 10
nb_ideas = 3
know_old = np.full((nb_nodes, nb_ideas), 0)

In [ ]:
know_old[0][2] = 1

know_old

In [ ]:
nb_nodes_per_idea = np.sum(know_old,axis=0)
nb_nodes_per_idea

In [ ]:
nb_ideas_per_node = np.sum(know_old,axis=1)
nb_ideas_per_node

In [ ]:
v = [0,1,0,1]
v.index(1,0,len(v))